<a href="https://colab.research.google.com/github/jermwatt/sqliteFST5demo/blob/master/sqlite_text_search_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.  A reminder that sqlite has decent text search capabilities

I always forget that sqlite has a simple-to-use [extension](https://www.sqlite.org/fts5.html) (that comes with Python's built in version) allows for decent full text search.  

This is a reminder notebook illustrating how to setup this full text search functionality on a small example dataset.  This is the "ASMR YouTube channels" [Kaggle dataset](https://www.kaggle.com/datasets/ulisesmontoyacanales/youtube-asmr-channels) - which some rich text columns detailing various facets of some ASMR YouTube channels (like a channel's `description`)

Let's get started.

Step 1: below we load in our dataset, printing the first few rows - here we see a number of text-based columns we may want to search over e.g., `title`, and `description`.

In [ ]:
# import standard libs
import os

# if on collab pull required dataet
if os.getenv("COLAB_RELEASE_TAG"): 
    from pathlib import Path
    import requests

    filename_url = 'https://raw.githubusercontent.com/jermwatt/sqliteFST5demo/master/ASMR_channels.csv'
    filename = filename_url.split('/')[-1]

    if Path(filename).is_file():
      print(filename,".csv already exists, skipping download")
    else:
      print("Downloading",filename)

    request = requests.get(filename_url)
    with open(filename, "wb") as f:
      f.write(request.content)

    print('download of',filename,'complete!')

# general imports 
import sys  
cwd = os.getcwd()
dataset_name = 'ASMR_channels.csv'
datapath = cwd + '/' + dataset_name
import pandas as pd

# read in dataset
df = pd.read_csv(dataset_name)

# examine head
df.head(2)

download of ASMR_channels.csv complete!


,channelId,customUrl,title,description,country,viewCount,subscriberCount,videoCount,madeForkids,creationDate
0,UCFmL725KKPx2URVPvH3Gp8w,@asmrglow,ASMR Glow,Here's everywhere you can find me: https://lin...,US,579354212,1720000,736,False,2016-10-24T08:22:54Z
1,UCvp9ypbbOQ2-vtuO-J3k-lw,@alexandriaasmr8317,Alexandria ASMR,ASMR - Autonomous sensory meridian response. I...,US,108539091,548000,295,False,2018-10-16T19:45:42Z


Let's use sqlite to create full text search capability over the `description` column.

This involves creating a special virtual table using Python's built-in `sqlite` library and `fts5` (full-text-search-5) built in extension - as shown below.

In [ ]:
# target filepath for sqlite db
import sqlite3, os
cwd = os.getcwd()
db_name = 'test.db'
dbpath = cwd + '/' + db_name

# connect to db
db = sqlite3.connect(dbpath)
cur = db.cursor()

# create a virtual table to house our text search
cur.execute('''
        CREATE VIRTUAL TABLE IF NOT EXISTS
            asmr_description_search 
        USING
            fts5(description, tokenize="porter unicode61");
    ''')

# dump description column into virtual table
cur.executemany('insert into asmr_description_search (description) values (?);', df[['description']].to_records(index=False))

# commit
db.commit()

We can now perform full text search our `description` column - as shown below - using  `sqlalchemy` connection.

In [ ]:
# define input keywords
search_phrase = 'i make asmr'

# execute full text search
cur.execute('''
    SELECT 
        * 
    FROM 
        asmr_description_search
    WHERE 
        description MATCH '" ''' + str(search_phrase) + ''' "'
    ORDER BY 
        rank
    LIMIT
        5
    ''')

rows = cur.fetchall()

for row in rows:
    print(row)

('I make ASMR videos 💖',)
('I make ASMR videos to relax you, give you tingles and help you sleep 😴',)
('Hi there! I make ASMR videos about music: music theory, music history, etc. Subscribe for more videos!',)
('Hi I make asmr crushing with my monster slippers, this will be mostly repurposed content from my TikTok',)
("Hello my lovely sweethearts💕 I'm Juliana, and I make ASMR videos to help you relax and fall asleep🥰",)
